# 도구를 사용한 Strands Agents와 AgentCore Memory (장기)

## 개요
이 노트북은 Strands와 AgentCore Memory를 사용하여 대화형 AI 에이전트에 장기 메모리 기능을 구현하는 방법을 보여줍니다. 단기 상호작용에서 중요한 정보를 추출하고 통합하여 에이전트가 시간이 지나면서 여러 대화 세션에 걸쳐 주요 세부사항을 기억할 수 있도록 하는 방법을 배울 수 있습니다.

## 튜토리얼 세부사항
**사용 사례:** 지속적인 메모리를 가진 요리 비서

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | 장기 대화형                                                                         |
| 에이전트 유형        | 요리 비서                                                                         |
| 에이전트 프레임워크   | Strands Agents                                                                   |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                      |
| 튜토리얼 구성요소     | AgentCore '사용자 선호도' 메모리 추출, 메모리 저장 및 검색을 위한 메모리 도구              |
| 예제 복잡도          | 초급                                                                             |

배울 내용:
- 장기 보존을 위한 추출 전략으로 AgentCore Memory 구성
- 이전 대화 기록으로 메모리 하이드레이트
- 대화 세션 간 개인화된 경험 제공을 위한 장기 메모리 사용
- Strands Agent Framework와 AgentCore Memory 도구 통합

## 시나리오 컨텍스트

이 튜토리얼에서는 매우 개인화된 레스토랑 추천을 제공하도록 설계된 요리 비서의 역할을 맡게 됩니다. AgentCore Memory의 장기 보존 및 자동 정보 추출을 활용하여 에이전트는 여러 대화에 걸쳐 식단 선택과 좋아하는 요리와 같은 사용자 선호도를 기억할 수 있습니다. 이러한 지속적인 메모리는 대화가 며칠 또는 몇 주에 걸쳐 진행되더라도 에이전트가 맞춤형 제안과 원활한 사용자 경험을 제공할 수 있게 합니다. 이 시나리오는 구조화된 메모리 조직과 구성 가능한 전략이 대화형 AI가 단기 회상을 넘어 진정으로 매력적이고 컨텍스트를 인식하는 상호작용을 만들 수 있도록 하는 방법을 보여줍니다.


## 아키텍처

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## 전제조건

이 튜토리얼을 실행하려면 다음이 필요합니다:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 권한이 있는 AWS 자격 증명
- Amazon Bedrock AgentCore SDK
환경 설정과 적절한 추출 전략을 가진 장기 메모리 리소스 생성을 시작해보겠습니다!

## 1단계: 환경 설정
이 노트북이 작동하도록 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것부터 시작해보겠습니다.

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import time
import logging
import time
from datetime import datetime

Amazon Bedrock 모델과 AgentCore에 대한 적절한 권한을 가진 지역과 역할을 정의합니다

In [ ]:
import os

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("culinary-memory")

region = os.getenv('AWS_REGION', 'us-west-2')

## 2단계: 장기 전략으로 메모리 생성

이 섹션에서는 장기 메모리 기능으로 구성된 메모리 리소스를 생성합니다. 이전의 단기 메모리 예제와 달리, 이 구현에는 통합된 정보 보존을 가능하게 하는 특정 메모리 전략이 포함되어 있습니다.

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)

memory_name = "CulinaryAssistant"
memory_id = None

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Long-Term Memory...")

    # We use a more descriptive name for our long-term memory resource
    memory_name = memory_name

    # Create memory with user preference strategy
    memory = client.create_memory_and_wait(
        name=memory_name,
        description="Culinary Assistant Agent with long term memory",
        strategies=[{
                    StrategyType.USER_PREFERENCE.value: {
                        "name": "UserPreferences",
                        "description": "Captures user preferences",
                        "namespaces": ["user/{actorId}/preferences"]
                    }
                }],
        event_expiry_days=7,
        max_wait=300,
        poll_interval=10
    )

    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### 장기 메모리 전략 이해

이 메모리 생성의 핵심 차이점은 **메모리 전략**의 추가입니다. 구성 요소를 살펴보겠습니다:

#### 1. 사용자 선호도 메모리 전략

이 전략은 대화에서 사용자 선호도를 자동으로 식별하고 추출합니다:

```python
"userPreferenceMemoryStrategy": {
    "name": "UserPreferences",
    "description": "Captures user preferences",
    "namespaces": ["user/{actorId}/preferences"]
}
```

#### 2. 메모리 네임스페이스

`namespaces` 매개변수는 추출된 정보가 저장되는 위치를 정의합니다:

```python
"namespaces": ["user/{actorId}/preferences"]
```

이 메모리 전략은 단순히 대화를 기억하는 것이 아니라 실제로 대화 내의 중요한 정보를 이해하고 미래 사용을 위해 정리하는 더 정교한 메모리 시스템을 만들어줍니다.

## 3단계: 이전 대화를 메모리에 저장

이 섹션에서는 단기 메모리를 하이드레이트하는 방법을 보여줍니다. 이는 백그라운드에서 장기 메모리 추출 프로세스를 자동으로 트리거합니다.

### 단기 메모리 하이드레이트

추출 전략으로 구성된 메모리 리소스에 대화를 저장할 때, 시스템은 추가 코드 없이 장기 보존을 위해 이 정보를 자동으로 처리합니다.

In [ ]:
actor_id = f"user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"foodie-{datetime.now().strftime('%Y%m%d%H%M%S')}"
namespace = f"user/{actor_id}/preferences"

In [ ]:
previous_messages = [
    ("Hi, I'm John", "USER"),
    ("Hi John, how can I help you with food recommendations today?", "ASSISTANT"),
    ("I'm looking for some vegetarian dishes to try this weekend.", "USER"),
    ("That sounds great! I'd be happy to help with vegetarian recommendations. Do you have any specific ingredients or cuisine types you prefer?", "ASSISTANT"),
    ("Yes, I really like tofu and fresh vegetables in my dishes", "USER"),
    ("Perfect! Tofu and fresh vegetables make for excellent vegetarian meals. I can suggest some stir-fries, Buddha bowls, or tofu curries. Do you have any other preferences?", "ASSISTANT"),
    ("I also really enjoy Italian cuisine. I love pasta dishes and would like them to be vegetarian-friendly.", "USER"),
    ("Excellent! Italian cuisine has wonderful vegetarian options. I can recommend pasta primavera, mushroom risotto, eggplant parmesan, or penne arrabbiata. The combination of Italian flavors with vegetarian ingredients creates delicious meals!", "ASSISTANT"),
    ("I spent 2 hours looking through cookbooks but couldn't find inspiring vegetarian Italian recipes", "USER"),
    ("I'm sorry you had trouble finding inspiring recipes! Let me help you with some creative vegetarian Italian dishes. How about stuffed bell peppers with Italian herbs and rice, spinach and ricotta cannelloni, or a Mediterranean vegetable lasagna?", "ASSISTANT"),
    ("Hey, I appreciate food assistants with good taste", "USER"),
    ("Ha! I definitely try to bring good taste to the table! Speaking of which, shall we explore some more vegetarian Italian recipes that might inspire you?", "ASSISTANT")
]

In [ ]:
print("\nHydrating short term memory with previous conversations...")

# Save the conversation history to short-term memory
initial = client.create_event(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    messages=previous_messages,
)
print("✓ Conversation saved in short term memory")

대화 메시지가 포함된 이벤트가 올바르게 저장되었는지 확인해보겠습니다.

In [ ]:
events = client.list_events(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    max_results=5
)
events

이 셀은 실행 중에 정보 메시지를 표시하도록 로깅 시스템을 구성하여 코드가 실행될 때 무엇이 일어나고 있는지 추적하는 데 도움을 줍니다.

### 백그라운드에서 일어나는 일

`create_event` 호출 후 다음이 자동으로 발생합니다:

1. **단기 저장**: 전체 대화가 원시 형태로 저장됨
2. **추출 트리거**: 메모리 시스템이 이 메모리에 UserPreference 전략이 구성되어 있음을 감지
3. **백그라운드 처리**: 추가 코드 없이 시스템이:
   - 선호도 지표에 대해 대화를 분석
   - "나는 채식주의자입니다" 및 "이탈리아 요리를 정말 좋아합니다"와 같은 진술을 식별
   - 이러한 선호도를 구조화된 데이터로 추출
4. **장기 통합**: 추출된 선호도가 구성된 네임스페이스(`user/{actorId}/preferences`)에 저장됨

추출과 통합은 자동으로 일어납니다. 우리는 에이전트와 대화를 유지하거나 단기 메모리를 하이드레이트하기만 하면 되고, 메모리 생성 시 구성한 전략이 나머지를 처리합니다.

이 자동 프로세스는 단기 대화 기록이 만료된 후에도 중요한 정보가 장기 메모리에 보존되도록 보장합니다.


## 장기 메모리 검색

이 섹션에서는 장기 메모리에 저장된 추출된 선호도에 액세스하는 방법을 탐구합니다. 대화 턴에 초점을 맞춘 단기 메모리 검색과 달리, 장기 메모리 검색은 추출되고 통합된 구조화된 정보에 액세스하는 데 초점을 맞춥니다.

### 장기 메모리에서 사용자 선호도 액세스

장기 메모리에서 정보를 검색하려면 메모리 생성 시 정의된 네임스페이스 구조를 사용합니다:


In [ ]:
# Adding a 30s wait to ensure the memory extraction has time to process the event
time.sleep(30)

try:
    # Query the memory system for food preferences
    food_preferences = client.retrieve_memories(
        memory_id=memory_id,
        namespace=namespace,
        query="food preferences",
        top_k=3  # Return up to 3 most relevant results
    )

    if food_preferences:
        print(f"Retrieved {len(food_preferences)} relevant preference records:")
        for i, record in enumerate(food_preferences):
            print(f"\nMemory {i+1}:")
            print(f"- Content: {record.get('content', 'Not specified')}")
    else:
        print("No matching preference records found.")

except Exception as e:
    print(f"Error retrieving preference records: {e}")

이 방법은 필요할 때 관련 메모리를 검색할 수 있게 해줍니다. 이제 기본사항을 배웠으니 에이전트를 구축해보겠습니다!

## 4단계: 에이전트 생성 
이 섹션에서는 네이티브 `agent_core_memory` 도구를 사용하여 AgentCore Memory를 Strands Agent와 통합하는 방법을 탐구합니다.

#### 장기 메모리 기능을 가진 에이전트 설정
메모리 지원 에이전트를 만들기 위해 Strands 프레임워크를 사용하고 AgentCore Memory 리소스에 연결합니다

In [ ]:
from strands import tool, Agent
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider

In [ ]:
system_prompt = f"""You are the Culinary Assistant, a sophisticated restaurant recommendation assistant.

PURPOSE:
- Help users discover restaurants based on their preferences
- Remember user preferences throughout the conversation
- Provide personalized dining recommendations

You have access to a Memory tool that enables you to:
- Store user preferences (dietary restrictions, favorite cuisines, budget preferences, etc.)
- Retrieve previously stored information to personalize recommendations

"""

In [ ]:
provider = AgentCoreMemoryToolProvider(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    namespace=namespace
)

agent = Agent(tools=provider.tools, model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",system_prompt=system_prompt)

이미 단기 및 장기 메모리를 채웠으므로 에이전트에서 직접 메모리를 검색해보겠습니다!

In [ ]:
agent("Give me restaurant recommendations in Irvine based on my food preferences")

에이전트는 사용자의 메모리를 검색하기 위해 retrieve_memory_records 메서드를 사용했을 것입니다.

훌륭합니다! 이제 AgentCore 장기 메모리에서 메모리를 검색할 수 있는 작동하는 Strands Agent를 가지게 되었습니다!

## 정리
이 노트북에서 사용한 리소스를 정리하기 위해 메모리를 삭제해보겠습니다.

In [ ]:
#client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)